Exercise 1 Simple neural network
Implement a simple neural network that predicts the class of iris plant basing on its parameters. 
Use 2 hidden layers. The layers should be fully connected with RELu activation function, except for the output layer, which should use softmax function.
 No regularisation or optimalisation are needed. No batching is needed. 


In [38]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
iris = datasets.load_iris()


In [39]:
print(iris.DESCR)


.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica

:Summary Statistics:

============== ==== ==== ======= ===== ====================
                Min  Max   Mean    SD   Class Correlation
============== ==== ==== ======= ===== ====================
sepal length:   4.3  7.9   5.84   0.83    0.7826
sepal width:    2.0  4.4   3.05   0.43   -0.4194
petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
============== ==== ==== ======= ===== ====================

:Missing Attribute Values: None
:Class Distribution: 33.3% for each of 3 classes.
:Cr

In [40]:
X = iris.data
y = iris.target

In [32]:
X.shape

(150, 4)

In [41]:
X_train , X_test , y_train, y_test = train_test_split(X,y , test_size=0.25 , random_state=42,shuffle=True)
train_labels = np.eye(3)[y] # one hot encoding 3 type of flowers  # full set 


In [24]:
class Neuron():
    def __init__(self,in_features,out_features):
        self.weights = np.random.uniform(low = - ((1 / in_features)**0.5) ,high= (1 / in_features)**0.5 , size = (in_features,out_features))    # as in PyTorch
        self.bias = np.random.uniform(low = - ((1 / in_features)**0.5) ,high= (1 / in_features)**0.5 , size = (1,out_features))
        
    
    


    

In [77]:
def hidden_unit(X , in_features, out_features):
    weights = np.random.uniform(low = - ((1 / in_features)**0.5) ,high= (1 / in_features)**0.5 , size = (in_features,out_features))    # as in PyTorch
    bias = np.random.uniform(low = - ((1 / in_features)**0.5) ,high= (1 / in_features)**0.5 , size = (1,out_features))
    
    return X @ weights + bias

def Relu_forward(X):
    return np.maximum(X,0)

def Relu_derivative(X):
    # basicly kill gradient if <=0
    X[X<=0] = 0
    # X[X>0] = 1
    return X

def Softmax_forward(X):
    X_exp = np.exp(X)
    row_sum = np.sum(X_exp,axis=1,keepdims=True)
    return X_exp/row_sum  
def Logloss(X,labels):
    # matrix after softmax 
    return -np.mean(labels * np.log(X))

def CrossEntropyBackward(Softmax_output,labels):
    # combining log loss and soft max gives nice derivative 
    # X is output of softmax and labels are true labels
    return Softmax_output - labels     




http://neuralnetworksanddeeplearning.com/chap2.html


In [100]:


# initializing neurons for layers
N_1 = Neuron(4,10)
N_2 = Neuron(10,10)
N_3 = Neuron(10,3)

X_1 = np.dot(X,N_1.weights)
A_1 = Relu_forward(X_1)

X_2 = np.dot(A_1,N_2.weights)
A_2 = Relu_forward(X_2)

X_3 = np.dot(A_2,N_3.weights)
A_3 = Softmax_forward(X_3)
loss = Logloss(A_3, train_labels)


N_3_error = A_3 - train_labels
N_3_weight_grad = np.dot(N_3_error.T,Relu_derivative(X_3))  # A_3 to co wpada do funckji aktywacji 
N_3_bias_grad = np.mean(N_3_error,axis=1,keepdims=True)

N_2_error = ((N_3.weights.T))
# N_2_weight_grad = np.dot(N_2_error,A_1)
# N_2_bias_grad = N_2_error

# N_1_error = 
# N_1_weight_grad = np.dot(N_1_error,X)
# N_1_bias_grad = N_1_error



In [101]:
X_3.shape

(150, 3)

In [97]:
N_3_weight_grad

array([[ 0.74497782,  0.75015403,  0.24975072],
       [-2.90621919, -2.40287636, -2.84332313],
       [ 2.16124137,  1.65272232,  2.5935724 ]])

In [92]:
np.mean(N_3_bias_grad,axis=1,keepdims=True)

(150, 1)

In [87]:
Relu_derivative(A_3).shape

(150, 3)

In [81]:
N_3_error.shape

(150, 3)

In [ ]:


# initializing neurons for layers
N_1 = Neuron(4,10)
N_2 = Neuron(10,10)
N_3 = Neuron(10,3)
for step in range(10):
    # forward pary
    # 1st layer
    X_1 = N_1.forward(X)
    X_1_Relu = Relu_forward(X_1)


    # 2nd layer
    X_2 = N_2.forward(X_1_Relu)
    X_2 = Relu_forward(X_2)


    # final layer
    X_3 = N_3.forward(X_2)

    X_softmax = Softmax_forward(X_3)

    loss = Logloss(X_softmax, train_labels)

    # now backward part

    Z_softmax = CrossEntropyBackward(X_softmax,train_labels)

    Z_3_weigths = X_2.T @ Z_softmax    # weights for final layer
    Z_3_bias = np.mean(Z_softmax , axis=1)           # + just plug forward gradient from previous layer  and calculate mean by rows 

    # we have relu implemented correctly for chain rule so we just now apply 

    Z_3_input = Z_softmax @ Z_3_weigths.T
        
    Z_2_relu = Relu_backward(Z_3_weigths)

    Z_2_weights = X_1_Relu @ Z_2_relu   # gradient for X_1 bias

    Z_2_bias = np.mean(Z_2_relu , axis=1)           # gradient for X_2 bias

    Z_1_relu = Relu_backward(Z_2_weights)

    Z_1_weights = X.T @ Z_1_relu    # gradient for X_1 weights
    Z_1_bias = np.mean(Z_1_relu , axis = 1)             # gradient for X_1 bias


    # gradients applied in this form will reset every loop 
    # for now let's hardcode learning rate to 0.01 so we need to update in reverse gradient direction 


    N_1.weights -= 0.01 * Z_1_weights
    N_1.bias -= 0.01 * Z_1_bias

    N_2.weights -= 0.01 * Z_2_weights
    N_2.bias -= 0.01 * Z_2_bias

    N_3.weights -= 0.01 * Z_3_weigths
    N_3.bias -= 0.01 * Z_3_bias


    print(f"Step: {step}, loss value: {loss}")


        
        

ValueError: operands could not be broadcast together with shapes (4,10) (4,3) (4,10) 